# Bootstrapping 

Generador de muestras Bootstrapping

In [ ]:
from enum import Enum
etiqueta = Enum('etiqueta', ["location", "fruta", "variedad", "tamaño", "plato", "luz", "plano", "angulo"], start=0)

traduccion_etiqueta = { "Rio": "Rioja",
                        "Mad": "Madrid",

                        "M": "Manzana",
                        "P": "Pera",
                        "N": "Naranja",
                        "PL": "Platano",

                        "Fuji": "Fuji",
                        "Golden": "Golden",
                        "Granny Smith": "Granny Smith",

                        "Spb": ["Sin_Plato", "Blanco"],
                        "Spo": ["Sin_Plato", "Oscuro"],
                        "Spm": ["Sin_Plato", "Mantel"],
                        "Ppb": ["Postre", "NULL"],
                        "Pgb": ["Grande", "NULL"],

                        "int": "Interior",
                        "ext": "Exterior",

                        "al": "Alejado",
                        "me": "Medio",
                        "ce": "Cercano",

                        "sup": "Superior",
                        "cen": "Central",
                        "ver": "Vertical"}

In [ ]:
def get_bin(true_values):
    
    grams = [100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350, 375]
    bins = [87.5, 112.5, 137.5, 162.5, 187.5, 212.5, 237.5, 262.5, 287.5, 312.5, 337.5, 362.5, 387.5]
    
    
    if type(true_values) == list:
        result = list()
        for e in true_values:
            l = len(bins)
            index = 0
            for i in range(l):
                try:
                    if bins[i] < float(e) and bins[i+1] > float(e):
                        index = i
                        result.append(grams[index])
                        break
                except:
                    print("Error:", e)
    else:
        l = len(bins)
        index = 0
        for i in range(l):
            try:
                if bins[i] < float(true_values) and bins[i+1] > float(true_values):
                    index = i
                    result = grams[index]
                    break
            except:
                print("Error:", true_values)
        
    
    return result

def get_images_info(path):
    
    f = path
    splited = path[0:-4].split('_') #Separamos por barraBaja, quitando .JPG
    splited[0] = splited[0].split('/')[-1]
    #print(splited)
    #Recopilar informacion imagen
    
    location = traduccion_etiqueta[splited[etiqueta.location.value]]
    fruta = traduccion_etiqueta[splited[etiqueta.fruta.value]]
    variedad = traduccion_etiqueta[splited[etiqueta.variedad.value]]
    tamaño = splited[etiqueta.tamaño.value].replace(',','.')
    luz = traduccion_etiqueta[splited[etiqueta.luz.value]]
    plano = traduccion_etiqueta[splited[etiqueta.plano.value]]
    angulo = traduccion_etiqueta[splited[etiqueta.angulo.value]]
    plato = traduccion_etiqueta[splited[etiqueta.plato.value]][0]
    superficie = traduccion_etiqueta[splited[etiqueta.plato.value]][1]
    
    if plato == "Sin_Plato":
        plato = plato + " " + superficie

    [location, fruta, variedad, tamaño, plato, luz, plano, angulo]
    return [f, location, fruta, variedad, get_bin(tamaño), plato, luz, plano, angulo]


In [ ]:
#Escribo en el fichero el path y tamaño extraido del path
from skimage import io
def write_file_df(writefile, df):
   
    # ESCRIBIR EN FICHERO
    print("Writing "+ writefile)
    with open('../data/dataset_files/bootstrapping/'+writefile, 'w') as fw: #Escribo en fichero Train
        for i in range(len(df.file)):
            path= df.file.iloc[i]
            tamaño = path.split('/')[-2].replace(',','.')

            #print(path, tamaño)

            #checking if it is a file
            if not os.path.isfile(path):
                raise Exception("File Not found: " + str(f))

            try:

                #_ = io.imread(path)
                string = str(path) + '*' + str(tamaño)+'\n'
                fw.write(string)

            except Exception as e:
                print("Exception: ", e, "image: ", path)

    fw.close()

In [ ]:
import os

import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

import imgclas
from imgclas import paths, config

from sklearn.model_selection import train_test_split

def get_df(dataset_file_path):
    df_test = pd.DataFrame(columns=['file', "location", "fruta", "variety", "size", "plate", "light", "shot", "angle"])
    f = open(dataset_file_path, "r")

    #Create pandas dataset
    for file in f.readlines():
        splited = file[0:-1].split('*')
        data = get_images_info(splited[0])

        df_test = df_test.append(pd.DataFrame([data], columns=['file', "location", "fruta", "variety", "size", "plate", "light", "shot", "angle"]), ignore_index=True)

    return df_test

In [ ]:

def gen_bootstrapping_samples(df, sample_size=1000, num_samples=1):
    
    diferents_sizes = df['size'].nunique()
    photos_per_size = int(sample_size / diferents_sizes)
    #print(photos_per_size)
    
    
    for i in range(0, num_samples):
        df_sample = pd.DataFrame(columns=['file', "location", "fruta", "variety", "size", "plate", "light", "shot", "angle"])
        for size in df['size'].unique():
            df_aux = df[(df['size']==size)]
            df_aux2 = df_aux.sample(n=photos_per_size, replace=True)
            df_sample = df_sample.append(df_aux2, ignore_index=True)
            #print("Tamaño: ", size, "df_aux: ", len(df_aux), "df_sample: ", len(df_aux2))
            
        if len(df_sample) < sample_size: #Si hay menos fotos, se añaden aleatoriamente de todo el test set
            extra_photos = sample_size - len(df_sample)
            df_aux = df.sample(n=extra_photos, replace=True)
            df_sample = df_sample.append(df_aux, ignore_index=True)

        #print(df_sample)
        write_file_df(f'bootstrapping_sample-{i}.txt', df_sample)
        
    return 0    

In [ ]:
df = get_df('/srv/image-classification-tf/data/dataset_files/test.txt')
gen_bootstrapping_samples(df, sample_size=1000, num_samples=100)